In [ ]:
import numpy as np
import pandas as pd

Für das nächste Beispiel verwenden wir einen Datensatz bei dem Brustkrebs klassifiziert wurde.

In [ ]:
# importieren eines Datensatzes
from sklearn.datasets import load_breast_cancer

data = load_breast_cancer()
df_data = pd.DataFrame(data.data, columns=data.feature_names)

In [ ]:
df_target = pd.DataFrame(data['target'])

Eine praktische Methode eines dataframes in pandas ist *describe*().

Sie zeigt alle Features und ihre statistischen Werte an.

In [ ]:
df_data.describe()

In [ ]:
# Aufteilen des Datensatzes in test & training set
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(df_data, df_target, test_size=0.33)

In [ ]:
from sklearn.svm import SVC
clf = SVC(gamma='scale')

In [ ]:
clf.fit(X_train, Y_train)

In [ ]:
prediction = clf.predict(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix

confusion_matrix(Y_test, prediction)

Was bedeutet der Output nun für uns?

Auf der linken Achse der Matrix stehen die realen Werte und auf der oberen die vorhergesagten.

Unser Klassifizierer klassifiziert also:
- 54 von 74 Leute ohne Brustkrebs richtig
- 112 von 114 Leute mit Brustkrebs richtig
    
Das bedeutet, dass 20 Leute ohne Brustkrebs als krank(False Positive) und 2 Leute mit Brustkrebs als gesund(False Negative) klassifiziert.

## Wie können wir unser Modell nun verbessern?

Es gibt mehrere Wege ein Machine Learning Model zu verbessern. Neben einem anderen Algorithmus gibt es vor allem im Preprocessing Wege es zu verbessern.
- Feature Engineering: dabei wählt man bestimmte Features aus und versucht anhand von Metriken (z.B. Abhängigkeit der Variablen) das Ergebnis zu verbessern
- Dimensionality Reduction: dabei wird der gesamte Feature Raum auf einen Unterraum abgebildet (Ein Beispiel hierfür ist PCA)
- Normalisieren: Alle Variablen auf den gleichen Werteraum normalisieren
- Hyperparameter optimieren

Wir werden uns im nächsten Abschnitt mit den letzteren drei Möglichkeiten beschäftigen.

In [ ]:
# zunächst importieren wir wieder die notwendigen Pakete
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA

Zunächst normalisieren wir alle Werte in unserem Datensatz auf einen Abschnitt zwischen 0 & 1.

Dadurch kann der Algorithmus besser konvergieren, da der Fehler nicht so schnell ansteigt.

In [ ]:
data_scaled = MinMaxScaler().fit_transform(df_data)
# the output from scikit-learn methods are numpy arrays so we need to transform it into a dataframe
df_data_scaled = pd.DataFrame(data_scaled)
df_data_scaled.columns = df_data.columns

In [ ]:
df_data_scaled.describe()

Nun sind alle unsere Werte auf den Bereich 0...1 normalisiert.
Versuchen wir jetzt noch einmal das selbe Modell zu trainieren.

In [ ]:
clf2 = SVC(gamma='scale')
# Split the datasets again
X_train2, X_test2, Y_train2, Y_test2 = train_test_split(df_data_scaled, df_target, test_size=0.33)
clf2.fit(X_train2, Y_train2)
prediction2 = clf2.predict(X_test2)

confusion_matrix(Y_test2, prediction2)

Wie wir sehen ist unsere Anzahl an False Positives sehr stark gesunken.

Das liegt daran, dass unsere Werte nun alle die gleiche Wertigkeit haben.

Bildlich bedeutet das, dass eine Messung die zwischen 0...1 liegt nun die gleiche Gewichtung hat, wie -100...100.

## Hyperparameter optimieren
Machine Learning Algorithmen haben, wie wir schon mit der Lernrate beim Perceptron gesehen haben Hyperparameter.

Diese können wir nutzen um unseren Algorithmus zu verbessern.

Die Attribute der SVC Klasse findet man unter: 
https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html#sklearn.svm.SVC


In [ ]:
# Ein Beispiel

clf3 = SVC(gamma='scale', degree=4)
# Split the datasets again
clf3.fit(X_train2, Y_train2)
prediction3 = clf3.predict(X_test2)

confusion_matrix(Y_test2, prediction3)

In [ ]:
#####################################################
#
#
# Versuche es hier einmal selbst
#


## Dimensionsreduktion
Nun nutzen wir Principal Component Analysis. Der Algorithmus sucht eine Funktion die Varianz des Datensatzes mit der so wenig Variablen wie möglich abzubilden.

In [ ]:
pca = PCA(n_components=10)
data_reduced = pca.fit_transform(df_data_scaled)

In [ ]:
df_data_reduced = pd.DataFrame(data_reduced)
df_data_reduced.head

import matplotlib.pyplot as plt
var_exp =  pca.explained_variance_ratio_
cum_var_exp = np.cumsum(var_exp)

plt.bar(range(1,11), var_exp, alpha=0.5, align='center',
        label='individual explained variance')
plt.step(range(1,11), cum_var_exp, where='mid',
         label='cumulative explained variance')
plt.ylabel('Explained variance ratio')
plt.xlabel('Principal components')
plt.legend(loc='best')
plt.show()

Wie wir sehen können beschreiben die ersten 5 Dimensionen u"ber 90% der Varianz im Datensatz (also auch des Informationsgehalts)
Wir können also die PCA auf lediglich 5 Komponenten reduzieren lassen.

In [ ]:
pca = PCA(n_components=3)
data_reduced = pca.fit_transform(df_data_scaled)

In [ ]:
clf_reduced = SVC(gamma='scale')
# Split the datasets again
X_train3, X_test3, Y_train3, Y_test3 = train_test_split(df_data_reduced, df_target, test_size=0.33)
clf_reduced.fit(X_train3, Y_train3)
prediction_red = clf_reduced.predict(X_test3)

confusion_matrix(Y_test3, prediction_red)

Wenn ihr es noch mit anderen Klassifizierern versuchen wollt, findet ihr auf dieser Seite ein Skript mit dem Vergleich mehrerer Klassifizierer.

https://scikit-learn.org/stable/auto_examples/classification/plot_classifier_comparison.html